In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#for data
!gdown --fuzzy https://drive.google.com/file/d/1L_HatfNfCfW0s25PPTh51A3HqUgvmvEz/view?usp=share_link -O animal_faces.zip

Downloading...
From: https://drive.google.com/uc?id=1L_HatfNfCfW0s25PPTh51A3HqUgvmvEz
To: /content/animal_faces.zip
100% 730M/730M [00:33<00:00, 22.1MB/s]


In [ ]:
!unzip animal_faces.zip

Streaming output truncated to the last 5000 lines.
  inflating: animal_faces/images/pixabay_dog_000311.jpg  
  inflating: animal_faces/images/pixabay_dog_000312.jpg  
  inflating: animal_faces/images/pixabay_dog_000313.jpg  
  inflating: animal_faces/images/pixabay_dog_000314.jpg  
  inflating: animal_faces/images/pixabay_dog_000315.jpg  
  inflating: animal_faces/images/pixabay_dog_000316.jpg  
  inflating: animal_faces/images/pixabay_dog_000317.jpg  
  inflating: animal_faces/images/pixabay_dog_000318.jpg  
  inflating: animal_faces/images/pixabay_dog_000319.jpg  
  inflating: animal_faces/images/pixabay_dog_000320.jpg  
  inflating: animal_faces/images/pixabay_dog_000321.jpg  
  inflating: animal_faces/images/pixabay_dog_000322.jpg  
  inflating: animal_faces/images/pixabay_dog_000323.jpg  
  inflating: animal_faces/images/pixabay_dog_000324.jpg  
  inflating: animal_faces/images/pixabay_dog_000325.jpg  
  inflating: animal_faces/images/pixabay_dog_000326.jpg  
  inflating: animal_f

In [ ]:
#for checkpoint
!gdown --fuzzy https://drive.google.com/file/d/1OuwiA2kPOxXJUAY7UdXnzz0RDvZWuy-M/view?usp=share_link -O /content/drive/MyDrive/checkpoint.pth


Downloading...
From: https://drive.google.com/uc?id=1OuwiA2kPOxXJUAY7UdXnzz0RDvZWuy-M
To: /content/drive/MyDrive/checkpoint.pth
100% 141M/141M [00:00<00:00, 150MB/s]


In [1]:
from tqdm import tqdm
import numpy as np
import PIL
import random
import torch
import torch.nn as nn
from torch.utils import data
from torchvision import datasets
import torchvision
from torchsummary import summary


torch.multiprocessing.set_start_method('spawn')
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Set default tensor type to cuda tensor if available
if device.type == "cuda":
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
print(torch.get_default_dtype())
# Set random seed for reproducibility
manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Using device: cpu
torch.float32
Random Seed:  999


In [2]:
# Root directory for dataset
dataroot = '/content/animal_faces/'

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 256

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 256

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 2000

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

transformation = torchvision.transforms.Compose([
    torchvision.transforms.Resize(image_size),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean,std)
])

In [3]:
# We can use an image folder dataset the way we have it setup.
# Create the dataset
dataset = datasets.ImageFolder(root=dataroot,
                           transform=transformation)
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers,
                                         generator=torch.Generator(device='cuda'))

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/animal_faces/'

In [ ]:
# Plot some training images
grid = torchvision.utils.make_grid(next(iter(dataloader))[0][:4]).permute([1, 2, 0])
image = grid.numpy()
image = ((image + 1) * 127.5).astype(np.uint8)
image = PIL.Image.fromarray(image, 'RGB')
image

In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [8]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 6, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 6),
            nn.ReLU(True),
            # state size. ``(ngf*6) x 8 x 8``
            nn.ConvTranspose2d( ngf * 6, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 16 x 16``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf * 2) x 32 x 32``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 64 x 64``
            nn.ConvTranspose2d( ngf, int(ngf/2), 4, 2, 1, bias=False),
            nn.BatchNorm2d(int(ngf/2)),
            nn.ReLU(True),
            # state size. ``(ngf/2) x 128 x 128``
            nn.ConvTranspose2d( int(ngf/2), nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 256 x 256``
        )

    def forward(self, input):
        return self.main(input)

In [9]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
netG.apply(weights_init)

# Print the model
summary(netG,(100,1,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1            [-1, 512, 4, 4]         819,200
       BatchNorm2d-2            [-1, 512, 4, 4]           1,024
              ReLU-3            [-1, 512, 4, 4]               0
   ConvTranspose2d-4            [-1, 384, 8, 8]       3,145,728
       BatchNorm2d-5            [-1, 384, 8, 8]             768
              ReLU-6            [-1, 384, 8, 8]               0
   ConvTranspose2d-7          [-1, 256, 16, 16]       1,572,864
       BatchNorm2d-8          [-1, 256, 16, 16]             512
              ReLU-9          [-1, 256, 16, 16]               0
  ConvTranspose2d-10          [-1, 128, 32, 32]         524,288
      BatchNorm2d-11          [-1, 128, 32, 32]             256
             ReLU-12          [-1, 128, 32, 32]               0
  ConvTranspose2d-13           [-1, 64, 64, 64]         131,072
      BatchNorm2d-14           [-1, 64,

In [10]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is ``(nc) x 256 x 256``
            nn.Conv2d(nc, int(ndf/2), 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf/2) x 128 x 128``
            nn.Conv2d(int(ndf/2), ndf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf) x 64 x 64``
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*2) x 32 x 32``
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*4) x 16 x 16``
            nn.Conv2d(ndf * 4, ndf * 6, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 6),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*6) x 8 x 8``
            nn.Conv2d(ndf * 6, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*8) x 4 x 4``
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [11]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
# like this: ``to mean=0, stdev=0.2``.
netD.apply(weights_init)

# Print the model
summary(netD, (3,image_size,image_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]           1,536
         LeakyReLU-2         [-1, 32, 128, 128]               0
            Conv2d-3           [-1, 64, 64, 64]          32,768
       BatchNorm2d-4           [-1, 64, 64, 64]             128
         LeakyReLU-5           [-1, 64, 64, 64]               0
            Conv2d-6          [-1, 128, 32, 32]         131,072
       BatchNorm2d-7          [-1, 128, 32, 32]             256
         LeakyReLU-8          [-1, 128, 32, 32]               0
            Conv2d-9          [-1, 256, 16, 16]         524,288
      BatchNorm2d-10          [-1, 256, 16, 16]             512
        LeakyReLU-11          [-1, 256, 16, 16]               0
           Conv2d-12            [-1, 384, 8, 8]       1,572,864
      BatchNorm2d-13            [-1, 384, 8, 8]             768
        LeakyReLU-14            [-1, 38

In [12]:
# Initialize the ``BCELoss`` function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = torch.optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = torch.optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# beta1 controls how much weight is given to the past gradient information in updating the first moment estimate. A higher value of beta1 means that the past gradients have more influence on the current estimate, while a lower value means that the current gradient has more influence.

In [16]:
try:
    loaded_dict = torch.load('../data/checkpoint.pth', map_location=torch.device('cpu'))
except FileNotFoundError as e:
    print(e)
    G_losses = []
    D_losses = []
    start = 1
else:
    netD.load_state_dict(loaded_dict['disc_w'])
    netG.load_state_dict(loaded_dict['gen_w'])
    optimizerD.load_state_dict(loaded_dict['disc_optim'])
    optimizerG.load_state_dict(loaded_dict['gen_optim'])
    D_losses = loaded_dict['history']['D_losses']
    G_losses = loaded_dict['history']['G_losses']
    start = loaded_dict['start']
    print('all data loaded')

all data loaded


In [ ]:
save_path_checkpoint = '/content/drive/MyDrive/checkpoint.pth'
# For each epoch
for epoch in range(start,num_epochs+1):
    # For each batch in the dataloader
    pbar = tqdm(dataloader,desc=f'Epoch {epoch}/{num_epochs}')
    for i, data in enumerate(pbar, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        pbar.set_postfix_str(f'Loss_D: {errD.item():.4f}\tLoss_G: {errG.item():.4f}\tD(x): {D_x:.4f}\tD(G(z)): {D_G_z1:.4f} / {D_G_z2:.4f}')

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

    # Check how the generator is doing by saving G's output on fixed_noise
    with torch.no_grad():
        fake = netG(fixed_noise).detach().cpu()
    image = torchvision.utils.make_grid(fake, padding=2, normalize=True).permute((1,2,0))
    image = image.numpy()
    image = ((image + 1) * 127.5).astype(np.uint8)
    image = PIL.Image.fromarray(image, 'RGB')
    image.save(f'/content/drive/MyDrive/images/image_{epoch}.png')

    torch.save({'disc_w':netD.state_dict(),'gen_w':netG.state_dict(),'disc_optim':optimizerD.state_dict(),'gen_optim':optimizerG.state_dict(),'history':{'G_losses': G_losses, 'D_losses': D_losses},'start':epoch+1},save_path_checkpoint)

In [18]:
with torch.no_grad():
    fake = netG(fixed_noise).detach().cpu()
image = torchvision.utils.make_grid(fake, padding=2, normalize=True).permute((1,2,0))
image = image.numpy()
image = ((image + 1) * 127.5).astype(np.uint8)
image = PIL.Image.fromarray(image, 'RGB')
image.save('current.png')